In [5]:
#Bibliotecas utilizadas
import pandas as pd 
from sklearn.model_selection import train_test_split  
from sklearn.neighbors import KNeighborsClassifier 
import matplotlib.pyplot as plt 
from sklearn import preprocessing 
from sklearn.metrics import classification_report

In [ ]:
#carregando dados 
covid_alagoas = pd.read_excel("covid_alagoas.xlsx")  

In [ ]:
#Exibição do dataset carregado
covid_alagoas.head() 

## Tratamento dos dados

In [ ]:
# excluindo colunas que não serão utilizadas 
del covid_alagoas["Unnamed: 0"] 
del covid_alagoas["Data do Óbito (Caso haja)"]"] 

# Renomeando colunas
covid_alagoas = covid_alagoas.rename(columns={'Situação do paciente confirmado (UTI, isolamento domiciliar, enfermaria) ?': 'Situacao'}) 
covid_alagoas = covid_alagoas.rename(columns={'MUNICÍPIO DE RESIDÊNCIA': 'Municipio'})    

#Onde tem NaN na coluna de comorbidades está sendo substituído por sem comorbidade
covid_alagoas['Comorbidades'] = covid_alagoas.Comorbidades.fillna('SEM COMORBIDADE')

# A coluna de SEXO está sendo rotulada como 1 para masculina e 2 para feminino
covid_alagoas.loc[covid_alagoas.SEXO == 'M','SEXO'] = 1 
covid_alagoas.loc[covid_alagoas.SEXO == 'F','SEXO'] = 2 
covid_alagoas.loc[covid_alagoas.SEXO == 'Feminino','SEXO'] = 2

### Selecionando os datasets


In [ ]:
#Aqui está sendo filtrado os dados referentes ao município de Maceió e Arapiraca 

covid_maceio = covid_alagoas.loc[covid_alagoas['Municipio'] == "MACEIÓ"] 
covid_arapiraca = covid_alagoas.loc[covid_alagoas['Municipio'] == "ARAPIRACA"]

In [ ]:
print("Tamanho dos dataframes:",len(covid_arapiraca),len(covid_maceio)) 

In [ ]:
covid_alagoas.head()

### Como há muitos mais dados de covid de Maceió em relação a Arapiraca, será selecionado uma amostra de mesmo tamanho de Arapiraca

In [ ]:
covid_maceio = covid_maceio.sample(30901)

In [ ]:
print("Tamanho dos dataframes agora:",len(covid_arapiraca),len(covid_maceio)) 

In [ ]:
# criando dicionário (rotulando) as situações
rotulo = dict(zip(covid_maceio.Situacao.unique(), covid_maceio.Situacao.unique()))   
rotulo

In [ ]:
#Rotulando comorbidades 

le = preprocessing.LabelEncoder() 
#le.fit_transform(covid_maceio['Comorbidades'].astype(str))
rotulo_comorbidades = le.fit_transform(covid_maceio['Comorbidades'].astype(str))  
covid_maceio['Comorbidades'] = le.fit_transform(covid_maceio['Comorbidades'].astype(str)) 

In [ ]:
rotulo_comorbidades.reshape(-1,1) 
# O array criado de rótulo de comorbidades será adicionado ao dataset 
covid_maceio.loc[:,'rotulo_comorbidades'] = rotulo_comorbidades

In [ ]:
covid_maceio.head()

In [ ]:
#selecionando as colunas que serão utilizadas para treinar e testar o modelo 

X = covid_maceio[['IDADE','SEXO','rotulo_comorbidades']]
y = covid_maceio['Situacao']

In [ ]:
#Selecionando conjunto de teste e treino, 70% doa dados serão usados para treino e 30% para teste 

#  70% / 30% 
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y,test_size=0.3, random_state=0) 

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5)  

In [ ]:
_2 = knn.fit(X_train_1, y_train_1) 

In [ ]:
# Estimando acurácia

est_2 = knn.score(X_test_1, y_test_1) 
est_2

In [ ]:
#pd.unique(covid_maceio[['Comorbidades', 'rotulo_comorbidades']].values.ravel('K')) 

In [ ]:
#Predição com base na idade, sexo e comorbidade respectivamente
prediction = knn.predict([[70,1,60]])
print("Situação do paciente: ",rotulo[prediction[0]])

### Trabalhando com o segundo dataset - Arapiraca 

In [ ]:
# criando dicionário (rotulando) as situações
rotulo = dict(zip(covid_arapiraca.Situacao.unique(), covid_arapiraca.Situacao.unique()))   
rotulo

In [ ]:
rotulo_comorbidades.reshape(-1,1) 
# O array criado de rótulo de comorbidades será adicionado ao dataset 
covid_arapiraca.loc[:,'rotulo_comorbidades'] = rotulo_comorbidades

In [ ]:
X_arapiraca = covid_arapiraca[['IDADE','SEXO','rotulo_comorbidades']]
y_arapiraca = covid_arapiraca['Situacao']

In [ ]:
#Selecionando conjunto de teste e treino
#  70% / 30% 
X_train, X_test, y_train, y_test = train_test_split(X_arapiraca, y_arapiraca,test_size=0.3, random_state=0) 

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5)  
treino = knn.fit(X_train, y_train) 

In [ ]:
# acurácia

estimativa = knn.score(X_test, y_test) 
print("Acurácia: ",estimativa)

In [ ]:
#pd.unique(covid_arapiraca[['Comorbidades', 'rotulo_comorbidades']].values.ravel()) 

In [ ]:
#Predição com base na idade, sexo e comorbidade respectivamente
prediction = knn.predict([[70,1,60]])
print("Situação do paciente: ",rotulo[prediction[0]])